In [8]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import numpy as np
from qiskit.primitives import StatevectorSampler
from qiskit_ibm_runtime import QiskitRuntimeService
import scipy as sp
import scipy.linalg



In [2]:
circuit = QuantumCircuit(2)

circuit.h(0)
circuit.cx(0,1)
circuit.draw()

┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘

In [5]:
# 1. A quantum circuit for preparing the quantum state |000> + i |111> / √2
qc = QuantumCircuit(3)
qc.h(0) # this is used to generate superposition
qc.p(np.pi/2 , 0) # this is to add quantum phase on the qubits 
qc.cx(0,1) # 0th-qubit-Controlled-NOT gate on 1st qubit
qc.cx(1,2) # 0th-qubit-Controlled-NOT gate on 2nd qubit
qc.draw()


┌───┐┌────────┐          
q_0: ┤ H ├┤ P(π/2) ├──■───────
     └───┘└────────┘┌─┴─┐     
q_1: ───────────────┤ X ├──■──
                    └───┘┌─┴─┐
q_2: ────────────────────┤ X ├
                         └───┘

In [6]:
# 2. Add the classical output in the form of measurement of all qubits
qc_measured = qc.measure_all(inplace=False)
job = StatevectorSampler().run([qc_measured] , shots=1000)
res = job.result()
print(f" > Counts: {res[0].data["meas"].get_counts()}")

#{'000': 497, '111': 503} which is 000 50% of the time and 111 50% of the time up to statistical fluctuations.

 > Counts: {'111': 508, '000': 492}
